In [27]:
import pandas as pd
import os
import re

precipitation_data = pd.DataFrame()
for file in os.listdir("lawas"):
    if re.search(r"rainfall-data\d+\.csv", file):
        df = pd.read_csv(f"lawas/{file}")
        precipitation_data = pd.concat([precipitation_data, df], ignore_index=True)

In [28]:
precipitation_data["DateTime"] = pd.to_datetime(precipitation_data["DateTime"],format="%Y%m%d %H%M")
precipitation_data = precipitation_data[["DateTime", "Rainfall (mm)"]]
precipitation_data.set_index("DateTime",inplace=True)
precipitation_data

,Rainfall (mm)
DateTime,
1998-07-13 12:00:00,1.0
1998-07-13 13:00:00,2.0
1998-07-13 14:00:00,0.0
1998-07-13 15:00:00,0.0
1998-07-13 16:00:00,0.0
...,...
2022-04-07 20:00:00,1.0
2022-04-07 21:00:00,0.0
2022-04-07 22:00:00,0.0


In [29]:
feature_data = pd.read_csv("lawas/rainfall-feature-wunderground.csv")
feature_data.rename(columns={"Time":"DateTime"},inplace=True)
feature_data["DateTime"] = pd.to_datetime(feature_data["DateTime"], format="%Y-%m-%d %H:%M:%S%z")
feature_data["DateTime"] = feature_data["DateTime"].dt.tz_localize(None)
feature_data.set_index("DateTime",inplace=True)
feature_data.drop(columns={"Wind Gust", "Precip."}, inplace=True)
feature_data.fillna({"Wind Speed":0}, inplace=True)
feature_data.ffill(inplace=True)
feature_data

,Temperature,Dew Point,Humidity,Wind,Wind Speed,Pressure,Condition
DateTime,,,,,,,
2002-08-01 00:00:00,79.0,75.0,89.0,E,3.0,29.69,Mostly Cloudy
2002-08-01 01:00:00,79.0,73.0,83.0,E,2.0,29.69,Mostly Cloudy
2002-08-01 02:00:00,77.0,75.0,94.0,W,2.0,29.66,Mostly Cloudy
2002-08-01 03:00:00,77.0,73.0,89.0,CALM,0.0,29.66,Mostly Cloudy
2002-08-01 04:00:00,79.0,73.0,83.0,NE,1.0,29.63,Mostly Cloudy
...,...,...,...,...,...,...,...
2022-04-07 19:00:00,79.0,75.0,89.0,W,3.0,29.66,Fair
2022-04-07 20:00:00,79.0,77.0,94.0,VAR,1.0,29.69,Fair
2022-04-07 21:00:00,79.0,77.0,94.0,VAR,2.0,29.72,Fair


In [30]:
lawas_rainfall = precipitation_data.join(feature_data)
lawas_rainfall.dropna(inplace=True)

In [31]:
lawas_rainfall.to_csv("lawas/lawas-rainfall.csv",index=True)